![Panel HighCharts Logo](https://raw.githubusercontent.com/MarcSkovMadsen/panel-highcharts/main/assets/images/panel-highcharts-logo.png)

# 📈 Panel HighGantt Reference Guide

The [Panel](https://panel.holoviz.org) `HighGantt` pane allows you to use the powerful [HighCharts](https://www.highcharts.com/) [Gantt Chart](https://www.highcharts.com/blog/products/gantt/) from within the comfort of Python 🐍 and Panel ❤️.

## License

The `panel-highcharts` python package and repository is open source and free to use (MIT License), however the **Highcharts js library requires a license for commercial use**. For more info see the Highcharts license [FAQs](https://shop.highsoft.com/faq).

## Parameters:

For layout and styling related parameters see the [Panel Customization Guide](https://panel.holoviz.org/user_guide/Customization.html).

* **``object``** (dict): The initial user `configuration` of the `chart`.
* **``object_update``** (dict) Incremental update to the existing `configuration` of the `chart`. 
* **``event``** (dict): Events like `click` and `mouseOver` if subscribed to using the `@` terminology.

___

# Usage

## Imports

You must import something from panel_highcharts before you run `pn.extension('highgantt')`

In [ ]:
import panel_highcharts as ph

Additionally you can specify extra Highcharts `js_files` to include or exclude. See the full list at [https://code.highcharts.com](https://code.highcharts.com)

In [ ]:
# ph.config.js_files(highcharts_exporting=False, highcharts_export_data=False, ...)

In [ ]:
from datetime import datetime as dt

import panel as pn
pn.extension('highgantt')

## Configuration

The `HighChart` pane is configured by providing a simple `dict` to the `object` parameter. For examples see the HighCharts [demos](https://www.highcharts.com/demo).

In [ ]:
# Source: https://jsfiddle.net/bxd210w4/26/
configuration = {
    "title": {"text": "Simple Gantt Chart"},
    "series": [
        {
            "name": "Project 1",
            "data": [
                {
                    "id": "s",
                    "name": "Start prototype",
                    "start": "2022/10/18",
                    "end": "2022/10/20",
                },
                {
                    "id": "b",
                    "name": "Develop",
                    "start": "2022/10/20",
                    "end": "2022/10/25",
                    "dependency": "s",
                },
                {
                    "id": "a",
                    "name": "Run acceptance tests",
                    "start": "2022/10/23",
                    "end": "2022/10/26",
                },
                {
                    "name": "Test prototype",
                    "start": "2022/10/27",
                    "end": "2022/10/29",
                    "dependency": ["a", "b"],
                },
            ],
        }
    ],
}

We need to transfer the datetime values in miliseconds and not as strings!

In [ ]:
EPOCH = dt.utcfromtimestamp(0)

def convert_to_miliseconds(value):
    s_dt = dt.strptime(value, "%Y/%m/%d")
    return int((s_dt - EPOCH).total_seconds() * 1000)


for point in configuration["series"][0]["data"]: # type: ignore
    point["start"] = convert_to_miliseconds(point["start"])
    point["end"] = convert_to_miliseconds(point["end"])

In [ ]:
chart = ph.HighGantt(object=configuration, sizing_mode="stretch_width", height=400)
chart

## Layout

In [ ]:
settings = pn.WidgetBox(
    pn.Param(
        chart,
        parameters=["height", "width", "sizing_mode", "margin", "object", "object_update", "event", ],
                widgets={"object": pn.widgets.LiteralInput, "object_update": pn.widgets.LiteralInput, "event": pn.widgets.StaticText},
        sizing_mode="fixed", show_name=False, width=250,
    )
)
pn.Row(settings, chart, sizing_mode="stretch_width", height=550)

Try changing the `sizing_mode` to `fixed` and the `width` to `500`.

## Updates

You can *update* the chart by providing a partial `configuration` to the `object_update` parameter.

In [ ]:
object_update = {
    "title": {"text": "Panel HighGantt Plan"},
}
chart.object_update=object_update

Verify that the `title` and `series` was updated in the chart above.

## Events

You can subscribe to chart events using an the `@` notation as shown below. If you add a string like `@name`, then the key-value pair `'channel': 'name'` will be added to the `event` dictionary.

In [ ]:
event_update = {
    "series": [
        {
            "allowPointSelect": "true",
            "point": {
                "events": {
                    "click": "@click;}",
                    "mouseOver": "@mouseOverFun",
                    "select": "@select",
                    "unselect": "@unselect",
                }
            },
            "events": {
                "mouseOut": "@mouseOutFun",
            }
        }
    ]
}
chart.object_update=event_update

Verify that you can trigger the `click`, `mouseOver`, `select`, `unselect` and `mouseOut` events in the chart above and that the relevant `channel` value is used.

## Javascript

You can use Javascript in the configuration via the `function() {...}` notation.

In [ ]:
# Todo: Insert Example. For now see the HighChart, HighStock or HighMap reference notebooks.
js_update = {}
chart.object_update=js_update

# App

Finally we can wrap it up into a nice app template.

In [ ]:
configuration["series"][0]={**configuration["series"][0], **event_update["series"][0]}
chart.object = configuration

In [ ]:
app = pn.template.FastListTemplate(
    site="Panel Highcharts",
    title="HighGantt Reference Example", 
    sidebar=[settings], 
    main=[chart]
).servable()

You can serve with `panel serve HighGantt.ipynb` and explore the app at http://localhost:5006/HighGantt.

![HighGantt Reference Guide](https://raw.githubusercontent.com/MarcSkovMadsen/panel-highcharts/main/assets/images/HighGanttApp.gif)